In [22]:
import pickle
import os
import numpy as np
import pandas as pd
import scipy.sparse as sp

In [2]:
with open('neg.pkl', 'rb') as f:
    data = pickle.load(f)

In [142]:
data[0]

array([ 709, 2741, 1912, 2135, 2659, 2208, 1684, 1516, 3739, 2734, 2952,
       1630, 4014, 3138, 3055, 4255, 1946, 4050, 3861, 2657, 1010, 3905,
       3276, 2637, 1394, 1341, 2650, 1050,  115, 4187, 1005, 3151,  202,
       2883, 3050, 1364, 1814, 1592, 2566, 4170, 3821, 1743, 2156, 4571,
       2752, 3164,  764, 2265, 3604, 1217,  113, 2122,   89, 2842,  804,
       3097, 1522, 2537, 1518,  480, 3572, 2844,  416, 4033, 4343, 2820,
       4030, 4136, 4619, 3205, 3424, 4156,  158, 1029, 3767, 3790, 3756,
        223, 1412, 4159,  671, 2186,  973,  526, 4671, 4483, 2589, 3851,
        578, 3586, 3002, 2419, 4060, 1037, 2166, 2681, 1820, 1696, 3574],
      dtype=int64)

In [171]:
def get_ub(task):
        with open(os.path.join('data/Youshu', 'user_bundle_{}.txt'.format(task)), 'r') as f:
            u_b_pairs = list(map(lambda s: tuple(int(i) for i in s[:-1].split('\t')), f.readlines()))

        indice = np.array(u_b_pairs, dtype=np.int32)
        values = np.ones(len(u_b_pairs), dtype=np.float32)
        u_b_graph = sp.coo_matrix(
            (values, (indice[:, 0], indice[:, 1])), shape=(8039, 4771)).tocsr()

        return u_b_pairs, u_b_graph

In [ ]:
def get_ub():
    with open('user_bundle.txt', 'r') as f:
        u_b_pairs = list(map(lambda s: tuple(int(i) for i in s[:-1].split('\t')), f.readlines()))

    indice = np.array(u_b_pairs, dtype=np.int32)
    values = np.ones(len(u_b_pairs), dtype=np.float32)
    u_b_graph = sp.coo_matrix(
        (values, (indice[:, 0], indice[:, 1])), shape=(8039, 4771)).tocsr()

    return u_b_pairs, u_b_graph

In [172]:
ub_pairs, ub_graph = get_ub('train')

In [193]:
neg = np.array([])
np.random.seed(2024)

for u in range(8039):
    neg_bun = np.setdiff1d(np.arange(4771), ub_graph.col[ub_graph.row == u])
    np.random.shuffle(neg_bun)
    neg.append(neg_bun[:100])

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [191]:
len(neg[0])

100

In [173]:
ub_graph = ub_graph.tocoo()

In [174]:
row_indices = np.where(ub_graph.row == 0)

In [192]:
for u in range(8039):
    row_indices = np.where(ub_graph.row == u)
    for x in neg[u]:
        if x in ub_graph.col[row_indices]:
            print(u, x)